In [1]:
import pandas as pd
import openpyxl as xl
import json

In [2]:
df = pd.read_excel("D://Abdelrahman//acc_project//Files QuickBooks//INTRO SOLAR//06 Jun 2024 SOLAR.xlsx", skiprows = [0, 1, 2])
df.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,NaN,Jun 2024,Total
1,Income,NaN,NaN
2,Sales of Product Income,NaN,0
3,Sales,4284649.34,4284649.34
4,Total Sales of Product Income,4284649.34,4284649.34
5,Total Income,4284649.34,4284649.34
6,Cost of Sales,NaN,NaN
7,Cost of sales,NaN,0
8,60010 Salaries (Operating),14874.99,14874.99
9,60080 Government fines,2238.74,2238.74


# hhhhhhhhhhhhhh

In [90]:
sections = [
    ("Income", "Cost of Sales", "Income_df"),
    ("Cost of Sales", "Gross Profit", "Cost_of_sales_df"),
    ("Gross Profit", "Gross Profit", "Gross_profit_df"),
    ("Expenses", "Other Expenses", "Expenses_df"),
    ("Other Expenses", "Net Earnings", "Other_Expenses_df"),
    ("Net Earnings", "Net Earnings", "Net_Earnings_df")
]

# Dictionary to hold DataFrames
dataframes = {}

for start_key, end_key, df_name in sections:
    start_idx = df.index[df['INTRO SOLAR...'] == start_key][0]
    end_idx = df.index[df['INTRO SOLAR...'] == end_key][0]

    # Create the DataFrame for each section
    if (start_key == "Gross Profit" and end_key == "Gross Profit") or (start_key == "Net Earnings" and end_key == "Net Earnings"):
        temp_df = df.iloc[start_idx : end_idx + 1, 0:2]
        temp_df.fillna(0, inplace=True)
    else:
        temp_df = df.iloc[start_idx + 1 : end_idx, 0:2]
        temp_df.fillna(0, inplace=True)
    
    # Set column names based on the section
    temp_df.columns = [start_key.replace(" ", "_"), "Total"]

    # Add the DataFrame to the dictionary
    dataframes[df_name] = temp_df

# Final dictionary to hold JSON structure
final_dict = {}

# Iterate over each DataFrame and convert to desired JSON format
for df_name, df in dataframes.items():
    final_dict[df_name] = [{row[df.columns[0]]: row['Total']} for _, row in df.iterrows()]

# Convert the final dictionary to JSON
json_data = json.dumps(final_dict, ensure_ascii=False, indent=4)

# Save to a JSON file
with open('financial_data.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_data)

C:\Users\Abdelrahman_advec\AppData\Local\Temp\ipykernel_6624\1734090923.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  temp_df.fillna(0, inplace=True)


# structured Data

In [99]:
import json

# Step 1: Read the JSON data from the file
with open('financial_data.json', 'r') as file:  # Replace 'data.json' with your file's name
    json_data = json.load(file)

def process_section(section):
    result = {}
    current_parent = None
    current_sub_parent = None
    
    for item in section:
        key = list(item.keys())[0]
        value = item[key]
        
        if key.startswith("   ") and not key.startswith("      "):  # 3 spaces
            current_parent = key.strip()
            result[current_parent] = {} if value == 0 else value
            current_sub_parent = None  # Reset sub-parent
        elif key.startswith("      "):  # 6 spaces
            sub_key = key.strip()
            if current_parent:
                if isinstance(result[current_parent], dict):
                    result[current_parent][sub_key] = value
                else:
                    result[current_parent] = {sub_key: value}
        else:  # No leading spaces, top-level entry
            result[key.strip()] = value
            current_parent = None  # Reset parent for top-level items

    return result

# Step 2: Process each section of the data
processed_data = {}
for key, section in json_data.items():
    processed_data[key] = process_section(section)

# Step 3: Convert the result to JSON format
structured_json = json.dumps(processed_data, indent=4)

# Optional: Save the structured data back to a new JSON file
with open('structured_data.json', 'w') as f:  # Output file name
    f.write(structured_json)

# Output the result to the console
print(structured_json)


{
    "Income_df": {
        "Sales of Product Income": {
            "Sales": 4284649.34
        },
        "Total Sales of Product Income": 4284649.34,
        "Total Income": 4284649.34
    },
    "Cost_of_sales_df": {
        "Cost of sales": {
            "60010 Salaries (Operating)": 14874.99,
            "60080 Government fines": 2238.74,
            "61030 Maintenance & Repairs": 3820.0,
            "Other costs of sales - COS": 0.0,
            "60070 Station Rent": 102832.0,
            "61215 Staff Accommodation": 1200.0,
            "67200 Safety & Security": 26970.0,
            "Hospitality - COS": 2000.0,
            "Rent Fees  - COS": 5000.0,
            "Total Other costs of sales - COS": 138002.0
        },
        "Total Cost of sales": 158935.73,
        "Total Cost of Sales": 158935.73
    },
    "Gross_profit_df": {
        "Gross Profit": 4125713.61
    },
    "Expenses_df": {
        "Financing Cost": {
            "Interest expense -  EBRD": 1993010.47
       